Author: Huangrui Chu

In [1]:
#I use keyboard toachieve the hit to talk function
#pip install keyboard

In [1]:
import IPython.display as ipd
import numpy as np
import pyaudio, wave

# Record the audio with specific duration

Actual speech capture. The captured speech signals must have 16-bit resolution, mono channel and be captured at a sampling rate of 16000
samples per second.

In [2]:
CHUNK = 1600 #frames_per_buffer set it to be 1600 so 10 chunk equal to 1 second
FORMAT = pyaudio.paInt16 #16-bit resolution
CHANNELS = 1 # mono channel 
RATE = 16000 #a sampling rate of 16000 samples per second
Expected_duration=1 # the length you expected to record unit: second
Expected_num_of_chunks= int(RATE/CHUNK*Expected_duration)# determine the number of chunk to record according to the Expected_duration
#the name to save the wavfile
#WAVE_OUTPUT_FILENAME = "Huangrui_Chu.wav"

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
print("* recording with format {}, {} channel(s),a sampling rate of {} samples per second and {} frames per buffer"
      .format(FORMAT,CHANNELS ,RATE,CHUNK))
print("start recording")
frames=[]
for i in range(0,Expected_num_of_chunks): 
    #read the data from sound card
    data = stream.read(CHUNK)
    frames.append(data)
print("recording over")      
array_frames=np.array(frames)
intframes=np.frombuffer(array_frames, np.int16)# decode
# waveData = intframes*1.0/max(abs(intframes))  # normalization

stream.stop_stream()
stream.close()
p.terminate()


* recording with format 8, 1 channel(s),a sampling rate of 16000 samples per second and 1600 frames per buffer
start recording
recording over


In [3]:
# frames=np.array(int_frame)
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)

# Endpointing

Endpointing, with hit-to-talk.

Hit to talk:

– Manually identify the beginning of an utterance

• E.g. hit a button/key and start speaking

– System automatically determines when the utterance ends


You may use one of the endpointing schemes mentioned in the lecture to find the trailing endpoint, or any other method you may come up with.

The endpointed segment must be written to file in ms wav or raw pcm format. 利用wave 存

In [4]:
#Here i install keyboard to achieve the hit to talk function
#pip install keyboard


## An Adaptive Endpointing Algorithm

• “forgetfactor” = forgetting factor; typical value >= 1   暂定设置成1 #这个factor 也是有点疑惑的

• “level” = smoothed signal level; initial value typically set to energy of first frame 

• “onset_threshold” = onset threshold 是一个定值 15 先试试,条件 onset比offset大

• “offset_threshold” = onset threshold-dynamic_range

• “dynamic_range” = 开始说话和结束说话的之间的能量差 设定成10，这样能拉开 onset offset的差距。

• “background” = background signal level; initially set to avg energy of first 10 frames 0-9 

• “adjustment” = adaptive update factor for background; higher values assume faster varying background. Typical value 0.05 (也叫做trackingfactor)

•  “alpha”= 在 A Two-Threshold Formula出现的控制 background的一个参数 如果我按照原本的代码写 当我被认为不在讲话的时候，设置alpha很小 （比如0.01）这样可以很快的把background拉到level，但是，很可能 下一个chunk后的level-background还是达不到onsetthreshold，那么 这个时候，background又会回到最近的level，然后继续判断为不在说话（其实人在说话） 所以这个alpha onset threshold dynamic range 怎么调整就非常的烦。 最后还是借鉴了 one threshold的方法 不用这个alpha， 表现反而好些。

In [2]:
def EnergyPerSampleInDecibel(audioframe):
    #应该是计算分贝
    #先转码，把他从byte 转成int16
    audioframe=np.frombuffer(np.array(audioframe),np.int16)
    #这边注意要把这个输入的data转变成int64的格式 这样在后续的分贝计算才不会出现 平方为负数（溢值了呗）
    np_audioframe=np.array(audioframe,dtype='int64')
    energy= 10*np.log10(np.sum(np_audioframe*np_audioframe))
    return energy

In [3]:
def classifyFrame(audioframe,level,forgetfactor,onset_threshold,background,adjustment):
    #An Adaptive Endpointing Algorithm
    current = EnergyPerSampleInDecibel(audioframe)
    isSpeech = False
    level = ((level * forgetfactor) + current) / (forgetfactor+ 1)
#     print("current energy {}".format(current))
#     print("current level is {}".format(level))
#     print("current background is {}".format(background))
    if (current < background):
        background = current
    else:
        background += (current - background) * adjustment
    if (level < background): 
        level = background
    if (level - background >onset_threshold): 
        isSpeech = True
#     print("updated background is {}".format(background))
    return isSpeech,level,background

In [4]:
def classifyFrame2(audioframe,inspeech,level,forgetfactor,onset_threshold,offset_threshold,background,adjustment,alpha):
    #A More Complex Algorithm with Two-Threshold Formula: onset_threshold,offset_threshold
    current = EnergyPerSampleInDecibel(audioframe)#current energy of the input audioframe
    if (current < background):
        background = current
    level = ((level * forgetfactor) + current) / (forgetfactor+ 1)
#     print("current energy {}".format(current))
#     print("current level is {}".format(level))
#     print("current background is {}".format(background))
    if inspeech:
        if (level-background < offset_threshold):
#             print("Are you talking? The difference between energy and background is small.")
#             print(level-background)
            inspeech=0
    else:
         if (level-background > onset_threshold):
                inspeech=1
    if inspeech:
        if level-background > onset_threshold:
             background += (level-background)* adjustment
#     else:
#         #如果不在说话 ，更新background,以防下面要说话。
#         background = alpha*background + (1-alpha)*level
#     print("updated background is {}".format(background))
#     if inspeech:
#         print("在讲话")
#     else:
#         print("没说话")
    return inspeech,level,background


In [8]:
import keyboard  #https://www.jianshu.com/p/8e508c6a05ce 参考这个做了 keyboard监听
import pyaudio, wave
def Endpointing_with_hit_to_talk():
    #hit to talk的 background初始化就有些难整 最好自然是按键的时候稍微停0.1 s，这样初始的background不会太大。
    #不然开口爆破音会让这个background在beginning的时候变化较大。
    #click "s" to start recording!
    forgetfactor= 1
    level= 0
    onset_threshold= 15
    dynamic_range=10
    offset_threshold= onset_threshold-dynamic_range
    background= 0
    adjustment= 0.05
    alpha=0.001
    beginning=True
    isSpeech=True # 默认不在说话
    time_silence=0
    expected_time_silence=0.5 #就你希望当判断你有多少s 没有说话 机器就停止记录
    
    CHUNK = 1600 #frames_per_buffer set it to be 1600 so 10 chunks equal to 1 second
    FORMAT = pyaudio.paInt16 #16-bit resolution
    CHANNELS = 1 # mono channel 单声道
    RATE = 16000 #a sampling rate of 16000 samples per second
    #the name to save the wavfile
    WAVE_OUTPUT_FILENAME = "Huangrui_Chu.wav"

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("* recording with format {}, {} channel(s),a sampling rate of {} samples per second and {} frames per buffer"
          .format(FORMAT,CHANNELS ,RATE,CHUNK))
    frames = []
    print("click \"s\" to start recording!")
    #在按下s之前后面的语句都不会执行
    keyboard.wait('s')
    print("please talking!")
    #keep recording data 就是按了 s 后 一直录入 直到判断为 end point
    while True:
         #read the data from sound card
        data = stream.read(CHUNK)
        #初始化level 和 background
        if beginning:
            #转码
            int_data=np.frombuffer(np.array(data),np.int16)
            #这边注意要把这个输入的data转变成int64的格式 这样在后续的分贝计算才不会出现 平方为负数（溢值了呗）
            int_data=np.array(int_data,dtype='int64')
            first_frame=int_data[0]
            first_ten_frames=int_data[0:5]
            print( first_ten_frames)
            level=first_frame*first_frame
            background=np.average(first_ten_frames*first_ten_frames)
            print("beginning level is {}".format(level))
            print("beginning background is {}".format(background))
            
            beginning=False
        #One threshold method
        isSpeech,level,background=classifyFrame(data,level,forgetfactor,onset_threshold,background,adjustment)
        #Two threshold method
        #isSpeech,level,background=classifyFrame2(data,isSpeech,level,forgetfactor,onset_threshold,offset_threshold,background,adjustment,alpha)
        #检查这个chunk有没有说话声
        if not isSpeech:
            #说明这个chunk没有声音
            time_silence+=CHUNK/RATE
        else:
            time_silence=0
        #录入数据
        frames.append(data)
        #当我salient足够长的时间，就停止录入
        if time_silence>=expected_time_silence:
            break
    print("Too long time no one speaking, stop recording!")
    #保存数据
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    #这边加了.to_bytes()是因为我之前把data按照 int64 储存了
    wf.writeframes(b''.join(frames))
    wf.close()
    print("Audio data saved as {}".format(WAVE_OUTPUT_FILENAME))
    stream.stop_stream()
    stream.close()
    p.terminate()
    return frames

In [9]:
new_frames=Endpointing_with_hit_to_talk()

* recording with format 8, 1 channel(s),a sampling rate of 16000 samples per second and 1600 frames per buffer
click "s" to start recording!
please talking!
[33 53 42 45 42]
beginning level is 1089
beginning background is 1890.2
Too long time no one speaking, stop recording!
Audio data saved as Huangrui_Chu.wav


In [10]:
array_frames=np.array(new_frames)# 把数据从list 转成 array，这样可以用numpy处理
intframes=np.frombuffer(array_frames, np.int16)# 转码
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)